<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/MLGymGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning
## Appliqué à [Gym.OpenAI]((https://gym.openai.com)

Voir la page d'[introduction à Gym](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPMLIntroGym.ipynb).

### Installation de gym

#### Outil AUTRE QUE COLAB (pyzo, jupyter lab, .....)

In [ ]:
!pip install gym

In [1]:
import gym
import numpy as np

#### <font color=red>Sous colab</font>
*Sous colab, l'affichage ne sera pas dynamique.. Il faudra passer soit par un rendu sous forme de plot (matplotlib), soit passer par une vidéo à enregistrer...*

***Solution pour affichage statique***<br>
*Il vous faut importer, seulement si vous utilisez colab, les packages suivants :*

In [ ]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install x11-utils

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

---
## Charger l' environnement
On utilise ici l'environnement `MountainCarContinuous-v0`.

In [2]:
env = gym.make('MountainCarContinuous-v0') 

---
### Les valeurs des caractéristiques
Pour `MountainCarContinuous-v0`, vous trouverez l'explication des valeurs [ici](https://github.com/openai/gym/wiki/MountainCarContinuous-v0).

| No | Observation | Min | Max |
|:--:|:--:|:--:|:--:|
| 0 | Position | -1.2 | 0.6 |
| 1 | Velocité | -0.07 | 0.07 |

Vérification : 

In [3]:
print(env.spec)
print("nombre et type d'actions = ", env.action_space) 
print("valeurs d'actions les plus elevées = ", env.action_space.high) 
print("valeurs d'actions les plus basses = ", env.action_space.low)  
print("nombre et type d'observations = ", env.observation_space) 
print("valeurs de  observations les plus basses = ", env.observation_space.low) 
print("valeurs de observations les plus elevées = ", env.observation_space.high) 

EnvSpec(MountainCarContinuous-v0)
nombre et type d'actions =  Box(1,)
valeurs d'actions les plus elevées =  [1.]
valeurs d'actions les plus basses =  [-1.]
nombre et type d'observations =  Box(2,)
valeurs de  observations les plus basses =  [-1.2  -0.07]
valeurs de observations les plus elevées =  [0.6  0.07]


### (Re)Initialiser l'environnement
`env.reset()` initialise les variables d'environnement. L'exécution est obligatoire après avoir chargé l'environnement. Ici ce sont la position de départ et la vitesse.
Pour avoir un environnement non déterministe, par défaut `env.reset()` retourne une position x aléatoire entre -0.4 et -0.6 et une vitesse initiale nulle : 

In [4]:
env.reset()

array([-0.40071066,  0.        ])

Mais ici nous souhaitons partir du même point (-0.5) à vitesse nulle à chaque test de séquence d'actions. Avant chaque évaluation de séquence d'actions, ***on n'utilisera pas*** `env.reset()` par la suite et on écrira donc : 

In [5]:
env.env.state= np.array([-0.5, 0])

---
### Les actions
- Dans `MountainCarContinuous-v0` : [-1.0] fait reculer le véhicule d'une force 1, [0] le met au point mort, [1.0] le fait avancer d'une force 1

- ***Exécuter une action :***
  - *exécution* : `observation, reward,done,info = env.step(action)`
    - `observation`: ici la position du mobile et sa vitesse ([x, v])
    - `reward`: utilité de l'état (récompense)
    - `done` : booléen (True si but atteint)
    - `info` : éventuelle information retournée par l'outil

In [6]:
env.env.state= np.array([-0.5, 0])
action = 1
observation, reward,done,info = env.step([float(action)])
print("x=", observation[0])
print("v=", observation[1])
observation, reward,done,info = env.step([float(action)])
print("x=", observation[0])
print("v=", observation[1])

x= -0.49867684300416926
v= 0.0013231569958307428
x= -0.49604042641180685
v= 0.002636416592362428


---
### La récompense (reward)
Le *reward* dépend l'environnement d'exécution.. Dans `MountainCarContinuous-v0`, il est de `100(si le but est atteint)-le nombre d'actions au carré` !!
Selon la méthode IA choisie, il peut être plus intéressant de définir sa popre fonction récompense 
(par exemple, on peut favoriser les séquences qui mène le plus loin possible..)


---

# Test d'algorithme génétique
## Test de programme évolutionnaire pour atteindre l'objectif
- Utiliser l'environnement `MountainCarContinuous-v0`, car il est plus rapide 
- Programmez (en python) un algo génétique avec initialement les éléments suivants : 
  - sequence : tableau de valeurs entières -1, 0 ou 1.
    - *(on pourrait prendre des valeurs réelles mais ici on se contentera d'entiers pour commencer)*
  - 1 point de croisement (au centre)
    - *(pour commencer, on pourrait proposer 2 points de croisement (au 1/3, et 2/3))*
  - taille sequence : taille_seq = 80
  - taille population initiale : taille_pop = 500
  - taux de gènes mutants dans une séquence : taux_mut_seq = 0.1
  - taux de séquences mutantes dans la population : taux_mut_pop = 0.2
  - nombre max de cycles de reproductions : nb_cycles = 3000


- Pour tester une séquence, il suffit de balayer le tableau et de lancer l'action correspondante..(en la transformant en un tableau de réel (1 -> [1.0])
- Les 4 meilleures séquences (11,22,33,44) se reproduisent entre elles, donnant donc 12 fils : 12, 13, 14, 21, 23, 24, 31, 32, 34, 41, 42, 43.
- Idéalement, un filtre doit permettre de supprimer les séquences identiques qui pourraient venir des croisements et des mutations.

- Tester votre algo sur le nb de générations et afficher le rendu toutes les 100 générations et bien sûr le rendu de la meilleure séquence finale.
  - (uniquement la position finale *colab*)
  - (sous forme d'animation sous autre outil)


- ***NB:*** 
  - Au plus simple il est possible d'utiliser une population (liste) de couples [sequence, utilite] où l'utilité serait la distance en x.

In [8]:
#exemple :  
env.env.state= np.array([-0.5, 0])
tab = np.random.randint(-1, 2, 3) 
couple = [tab, 0]
print(couple)
for a in couple[0]:
    obs, reward, done, info = env.step([a])
print("arrive en x=", round(obs[0],2), ", fini=", done)
couple[1] = obs[0]
print("couple [actions, arrivee] = ", couple)

[array([-1, -1,  0]), 0]
arrive en x= -0.51 , fini= False
couple [actions, arrivee] =  [array([-1, -1,  0]), -0.5084984048091237]


### Bonus : 
- Adapter la récompense pour favoriser les séquences qui atteignent le but en un minimum d'actions utiles (0 étant considéré comme une action inutile)